In [1]:
import pandas as pd
import datetime
import glob

## Wrist and Hip Filter With DO_log_final Dates

In [2]:
#added column for id
def read_in_data(location, path):
    files = glob.glob(path + location + "_agg/*")
    df = pd.read_csv(files[0])
    filename = files[0].split("agg")[-1][1:-4]
    df["file"] = filename
    df['id'] = filename.split('_')[1][:4]
    for file in files[1:]:
        new_df = pd.read_csv(file)
        filename = file.split("agg")[-1][1:-4]
        new_df["file"] = filename
        new_df['id'] = filename.split('_')[1][:4]
        df = df.append(new_df)
    return df

In [3]:
df_wrist = read_in_data("wrist", '')
df_hip = read_in_data("hip", '')

## Merge With Groundtruth based on ID and Date

In [25]:
groundtruth = pd.read_csv('groundtruth.csv').drop('Unnamed: 0', axis = 1)

In [33]:
groundtruth['date'] = groundtruth['time'].str.split(' ').apply(lambda x: x[0])
groundtruth['year'] = groundtruth['date'].str.split('/').apply(lambda x: x[2])
groundtruth['month'] = groundtruth['date'].str.split('/').apply(lambda x: x[0])
groundtruth['day'] = groundtruth['date'].str.split('/').apply(lambda x: x[1])
groundtruth['datetime'] = pd.to_datetime(groundtruth[['year', 'month', 'day']]).astype(str)
groundtruth['actual_datetime'] = groundtruth['datetime'] + ' ' + groundtruth['actual_time']

In [38]:
groundtruth.drop(['date', 'year', 'month', 'day', 'datetime'], inplace = True, axis = 1)

In [39]:
groundtruth.head()

,time,id,coding,primary_behavior,primary_posture,primary_upperbody,primary_intensity,secondary_behavior,secondary_posture,secondary_upperbody,...,posture_coding,type,start.time,broad_activity,detailed_activity,walking_running_bouts,updated_activity,act_type,act_type_broad,actual_datetime
0,8/2/2017 18:30,AM07,sedentary,"LES- screen basedentary leisure time (TV, vide...",SB-sitting,no,sedentary,None,None,None,...,SB-sitting,E-7,8/2/2017 11:30,sit/stand,sit/lie,NaN,sit/lie,LES-screen,LES,2017-08-02 11:30:13
1,8/2/2017 18:30,AM07,sedentary,"LES- screen basedentary leisure time (TV, vide...",SB-sitting,no,sedentary,None,None,None,...,SB-sitting,E-7,8/2/2017 11:30,sit/stand,sit/lie,NaN,sit/lie,LES-screen,LES,2017-08-02 11:30:14
2,8/2/2017 18:30,AM07,sedentary,"LES- screen basedentary leisure time (TV, vide...",SB-sitting,no,sedentary,None,None,None,...,SB-sitting,E-7,8/2/2017 11:30,sit/stand,sit/lie,NaN,sit/lie,LES-screen,LES,2017-08-02 11:30:15
3,8/2/2017 18:30,AM07,sedentary,"LES- screen basedentary leisure time (TV, vide...",SB-sitting,no,sedentary,None,None,None,...,SB-sitting,E-7,8/2/2017 11:30,sit/stand,sit/lie,NaN,sit/lie,LES-screen,LES,2017-08-02 11:30:16
4,8/2/2017 18:30,AM07,sedentary,"LES- screen basedentary leisure time (TV, vide...",SB-sitting,no,sedentary,None,None,None,...,SB-sitting,E-7,8/2/2017 11:30,sit/stand,sit/lie,NaN,sit/lie,LES-screen,LES,2017-08-02 11:30:17


In [44]:
df_wrist.head()

,Accelerometer X,Accelerometer Y,Accelerometer Z,actual_datetime,file,id
0,0.250950,0.920588,-0.327150,2017-10-02 10:15:00,AG_AM01WRT (2017-10-02)AGGREGATE,AM01
1,0.251600,0.918650,-0.331750,2017-10-02 10:15:01,AG_AM01WRT (2017-10-02)AGGREGATE,AM01
2,0.225587,0.912912,-0.381100,2017-10-02 10:15:02,AG_AM01WRT (2017-10-02)AGGREGATE,AM01
3,-0.042238,0.976525,-0.147313,2017-10-02 10:15:03,AG_AM01WRT (2017-10-02)AGGREGATE,AM01
4,-0.287288,0.895525,-0.421950,2017-10-02 10:15:04,AG_AM01WRT (2017-10-02)AGGREGATE,AM01


In [41]:
groundtruth.actual_datetime.unique()

array(['2017-08-02 11:30:13', '2017-08-02 11:30:14',
       '2017-08-02 11:30:15', ..., '2017-08-02 11:01:23',
       '2017-08-02 11:01:24', '2017-08-02 11:01:25'], dtype=object)

In [42]:
df_wrist.actual_datetime.unique()

array(['2017-10-02 10:15:00', '2017-10-02 10:15:01',
       '2017-10-02 10:15:02', ..., '2018-05-19 13:14:48',
       '2018-05-19 13:14:49', '2018-05-19 13:14:50'], dtype=object)

In [47]:
wrist_merged = df_wrist.merge(groundtruth, on = ['actual_datetime', 'id'], how = 'inner')
hip_merged = df_hip.merge(groundtruth, on = ['actual_datetime', 'id'], how = 'inner')

In [48]:
print('percent of raw data')
len(hip_merged) / len(df_hip), len(wrist_merged) / len(df_wrist)

(0.01680762975663259, 0.017852820856866774)

In [49]:
hip_merged.to_csv('hip_merged.csv', index = False)
wrist_merged.to_csv('wrist_merged.csv', index = False)

## DO_log_final_df filter (useless)

In [4]:
DO_log_final_df = pd.read_csv('DO_log_final.csv')

,id,obs,type,start_month,start_day,start_year,start_time,stop_time,duration,session
1,AM01,DO1,H,10,6,2017,16:43:57,18:45:00,2:01:03,2
2,AM02,DO1,H,7,24,2017,13:17:10,15:17:32,2:00:22,1
6,AM03,DO2,H,7,27,2017,13:01:29,15:00:39,1:59:10,2
12,AM07,DO1,H,8,2,2017,9:01:18,11:01:27,2:00:09,1
40,AM19,DO1,H,11,9,2017,11:33:03,13:34:00,2:00:57,1


In [7]:
def datetime_filter(location, data, log):
    log = log[log['type'] == location[0].upper()]
    
    log['date'] = pd.to_datetime(log[['start_day','start_month','start_year']]
                   .astype(str).apply(' '.join, 1), format='%d %m %Y').astype(str)
    log['start_time'] = pd.to_datetime(log['start_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    log['stop_time'] = pd.to_datetime(log['stop_time'], format = '%H:%M:%S').apply(lambda x: x.time())
    
    data['actual_datetime'] = pd.to_datetime(data['actual_datetime'], format = '%Y-%m-%d %H:%M:%S')
    data['actual_date'] = data['actual_datetime'].apply(lambda x: x.date()).astype(str)
    data['actual_time'] = data['actual_datetime'].apply(lambda x: x.time())
    
    joined = pd.merge(data, log, how = 'left', left_on = 'actual_date', right_on = 'date')
    
    joined = joined[(joined['actual_time'] >= joined['start_time']) & 
                    (joined['actual_time'] <= joined['stop_time'])]
    
    return joined[['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'actual_datetime', 'file']]

In [8]:
hip_filtered = datetime_filter('hip', df_hip, DO_log_final_df)
wrist_filtered = datetime_filter('wrist', df_wrist, DO_log_final_df)

<ipython-input-7-392f920baa60>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['date'] = pd.to_datetime(log[['start_day','start_month','start_year']]
<ipython-input-7-392f920baa60>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['start_time'] = pd.to_datetime(log['start_time'], format = '%H:%M:%S').apply(lambda x: x.time())
<ipython-input-7-392f920baa60>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [9]:
hip_filtered['actual_datetime'].apply(lambda x: x.date()).unique()

array([datetime.date(2017, 10, 6), datetime.date(2017, 7, 24),
       datetime.date(2017, 7, 27), datetime.date(2017, 8, 2),
       datetime.date(2017, 11, 9), datetime.date(2017, 12, 2),
       datetime.date(2018, 2, 17), datetime.date(2018, 2, 24),
       datetime.date(2018, 5, 18)], dtype=object)

In [10]:
DO_log_final_df[DO_log_final_df['type'] == 'H'][['start_month', 'start_day', 'start_year']]

,start_month,start_day,start_year
1,10,6,2017
2,7,24,2017
6,7,27,2017
12,8,2,2017
40,11,9,2017
42,12,2,2017
50,2,17,2018
53,2,24,2018
54,2,24,2018
56,5,18,2018


In [11]:
hip_filtered.to_csv('hip_filtered.csv', index = False)
wrist_filtered.to_csv('wrist_filtered.csv', index = False)